# Utils

> Utils function used in the environment.

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import numpy as np
import pandas as pd
import toml

In [ ]:
#| export
def read_toml(  config_file= "Config.toml" # Path to the config file
):
    "Read setting file. The File containes the relative path to source and target."
    config= toml.load(config_file)
    return config
    

In [ ]:
# Here an example on how to structure a config.toml file.
config  = read_toml()
config

{'source': {'path': 'data/source',
  'filename_mesh': 'bone.key',
  'filename_landmarks': 'landmarks_source.csv'},
 'target': {'path': 'data/target',
  'filename_landmarks': 'landmarks_target.key'}}

In [ ]:
#| export
def _merge_path(path1:str , path2: str)-> str:
    "Join one or more path components. Return the join path in str type."
    return os.path.join(path1,path2)

In [ ]:
_merge_path(config['source']['path'],config['source']['filename_mesh'])

'data/source/bone.key'

In [ ]:
#| export
def multiple_targets():
    "This function checks if the there are multiple target subject."
    config = read_toml()
    path= config['target']['path']
    filename = config['target']['filename_landmarks']
    list_target= [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]
    if not list_target:
        assert os.path.isfile(_merge_path(path, filename)), "No file with Landmarks"
        return False
    else:
        return list_target


In [ ]:
multiple_targets()

['subj_1', 'subj_2']

In [ ]:
#| export
def read_k_file(path_to_file: str # File to read from 
                 )-> pd.DataFrame: # Dataframe of shape [n_nodes, [ID,x,y,z]]
    "This function read *NODES from .k/.key file."
    id_list = []
    node_coords = []
    find_nodes = False
    
    #assert os.path.exists(path_to_file), "Wrong path. Path not existent. Check the name/extension of the file."
    
    with open(path_to_file, 'r') as fp:
        for _, line in enumerate(fp):
            if line.startswith('*NODE'):
                find_nodes = True
                continue
            if find_nodes and line.startswith('*'):
                break
            if find_nodes:
                node_id = int(line[:8]) 
                node_x = float(line[8:24])
                node_y = float(line[24:40])
                node_z = float(line[40:56])
                id_list.append(node_id)
                node_coords.append([node_x, node_y, node_z])   
                 
    assert node_coords, "Nodes have NOT been read."
    
    df = pd.concat([pd.DataFrame(id_list), pd.DataFrame(node_coords)],axis=1)
    df.columns = ['Label - node id','x','y','z']
    return df

In [ ]:
read_k_file(_merge_path(config['source']['path'],config['source']['filename_mesh']))

,Label - node id,x,y,z
0,591310,-181.550415,-145.430511,404.375488
1,3591330,-43.251625,-196.187363,194.874634
2,3591332,-37.076435,-219.493759,205.082413


In [ ]:
#| export
def read_csv_file(path_to_file: str)-> pd.DataFrame:
    "This function read csv files in format id, x, y,z. to Pandas DataFrame"
    with open(path_to_file) as fp:
            df = pd.read_csv(fp, header=None)
            df= df.iloc[:,:4]
            df.columns = ['Label - node id','x','y','z']
    return df

In [ ]:
df = read_csv_file(_merge_path(config['source']['path'],config['source']['filename_landmarks']))
df

,id,x,y,z
0,1,100.001907,110.13913,-100.980621


In [ ]:
#| export
def from_df_to_np(df: pd.DataFrame)->np.ndarray:
    "This function transform a pd.Dataframe of shape [ID, x,y,z] to numpy array of coordinates [x,y,z]"
    if len(df.columns) == 4:
        return df.iloc[:,1:4].values
    
    elif len(df.columns) == 3:
        return df.values
    else:
        raise Exception("Invalid number of column in the Dataframe. Expected either 3 or 4 columns. ")

In [ ]:
from_df_to_np(df) #return np.array

array([[ 100.00190735,  110.13912964, -100.98062134]])

In [ ]:
#| export
def to_ls_dyna(number):
    return '{:16f}'.format(number)

In [ ]:
to_ls_dyna(6.8)

'        6.800000'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()